In [ ]:
# conda install -c plotly plotly=4.14.3

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# DEFINO FUNCIÓN PARA OBTENER ETIQUETAS DE TIEMPO

def get_TimeLabels(DataFrame, timeTag = 'Time', not_list = 'None'):    
    """
    This function returns the timestamps names of the DataFrame browsing the time Tag substring in the column names of the DataFrame 
    not_list: Column name that will be not in the list
    """
    
    TimeLabels=[]
    for col_name in list(DataFrame.columns):
        if 'Time' in col_name:
            TimeLabels.append(col_name)
    if not_list != 'None':
        TimeLabels.remove(not_list)
    return TimeLabels

In [3]:
def dataAlign(signal, samplePeriod):
    import numpy as np
    new_signal=[]
    for i in range(0,round(len(signal.dropna()))+1):
        new_signal.append(signal[i])
        
        for j in range(0,samplePeriod-1):
            new_signal.append(float('Nan'))
    
    if len(new_signal) < len(signal):
        for j in range(0,(len(signal)-len(new_signal))):
            new_signal.append(float('Nan'))

    return new_signal

In [6]:
# DEFINO ESTA FUNCIÓN PARA ELIMINAR DE LOS NOMBRES DEL DATAFRAME SUBCADENAS
def str_delete(data, string):
    for i in data.columns:
        if string in i:
            data.rename(columns = {i : i.replace(string,'')}, inplace = True)
    return data

In [7]:
data=pd.read_csv(r'C:\Users\alexb\Documents\Facultad\UBA\Modelo_PGSM_3T\Datos\20210811 - Segunda corrida de datos\20210811_Variables_Modelo_Ext_L4.csv', delimiter=';', sep='\n',encoding = 'utf-16', decimal=',')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399452 entries, 0 to 399451
Data columns (total 56 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Humedad_Semilla Time            399452 non-null  object 
 1   Humedad_Semilla ValueY          399452 non-null  float64
 2   T_Mat_entrada Time              399452 non-null  object 
 3   T_Mat_entrada ValueY            399452 non-null  float64
 4   T_Solvente Time                 399452 non-null  object 
 5   T_Solvente ValueY               399452 non-null  float64
 6   Q_Solvente Time                 199725 non-null  object 
 7   Q_Solvente ValueY               199725 non-null  float64
 8   T_Lam_9 Time                    39944 non-null   object 
 9   T_Lam_9 ValueY                  39944 non-null   float64
 10  Planta_Marcha Time              13 non-null      object 
 11  Planta_Marcha ValueY            13 non-null      float64
 12  Ritmo_Molienda T

Guardo las etiquetas te tiempo (timestamp) de cada variable para luego poder borrarlas del dataset. Dejo solamente Ritmo_Molienda Time para usarla com índice.

In [9]:
Etiquetas_time = get_TimeLabels(data, not_list = "Ritmo_Molienda Time")

In [10]:
# Etiquetas a eliminar
data[Etiquetas_time]

,Humedad_Semilla Time,T_Mat_entrada Time,T_Solvente Time,Q_Solvente Time,T_Lam_9 Time,Planta_Marcha Time,Q_Miscela Time,Vel_Extractor Time,Conc_Miscela Time,Valid_NIR_Salida_Ext Time,...,Conc_P3-5 Time,Conc_P3-6 Time,Conc_P3-7 Time,Conc_P3-8 Time,Conc_P3-9 Time,Conc_P3-10 Time,Conc_P3-11 Time,Conc_P1 Time,TT-35-GI Time,TT-35-GO Time
0,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,26/07/2021 8:51:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 7:08:02,...,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00
1,25/07/2021 0:00:01,25/07/2021 0:00:01,25/07/2021 0:00:01,25/07/2021 0:00:02,25/07/2021 0:00:10,26/07/2021 8:52:00,25/07/2021 0:00:02,25/07/2021 0:00:01,25/07/2021 0:00:01,25/07/2021 7:08:50,...,25/07/2021 0:00:10,25/07/2021 0:00:10,25/07/2021 0:00:10,25/07/2021 0:00:10,25/07/2021 0:00:10,25/07/2021 0:00:10,25/07/2021 0:00:10,25/07/2021 0:00:10,25/07/2021 0:00:01,25/07/2021 0:00:05
2,25/07/2021 0:00:02,25/07/2021 0:00:02,25/07/2021 0:00:02,25/07/2021 0:00:04,25/07/2021 0:00:20,26/07/2021 10:40:00,25/07/2021 0:00:04,25/07/2021 0:00:02,25/07/2021 0:00:02,25/07/2021 11:32:05,...,25/07/2021 0:00:20,25/07/2021 0:00:20,25/07/2021 0:00:20,25/07/2021 0:00:20,25/07/2021 0:00:20,25/07/2021 0:00:20,25/07/2021 0:00:20,25/07/2021 0:00:20,25/07/2021 0:00:02,25/07/2021 0:00:10
3,25/07/2021 0:00:03,25/07/2021 0:00:03,25/07/2021 0:00:03,25/07/2021 0:00:06,25/07/2021 0:00:30,26/07/2021 10:41:00,25/07/2021 0:00:06,25/07/2021 0:00:03,25/07/2021 0:00:03,25/07/2021 11:32:50,...,25/07/2021 0:00:30,25/07/2021 0:00:30,25/07/2021 0:00:30,25/07/2021 0:00:30,25/07/2021 0:00:30,25/07/2021 0:00:30,25/07/2021 0:00:30,25/07/2021 0:00:30,25/07/2021 0:00:03,25/07/2021 0:00:15
4,25/07/2021 0:00:04,25/07/2021 0:00:04,25/07/2021 0:00:04,25/07/2021 0:00:08,25/07/2021 0:00:40,26/07/2021 14:05:00,25/07/2021 0:00:08,25/07/2021 0:00:04,25/07/2021 0:00:04,26/07/2021 8:51:00,...,25/07/2021 0:00:40,25/07/2021 0:00:40,25/07/2021 0:00:40,25/07/2021 0:00:40,25/07/2021 0:00:40,25/07/2021 0:00:40,25/07/2021 0:00:40,25/07/2021 0:00:40,25/07/2021 0:00:04,25/07/2021 0:00:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399447,29/07/2021 14:57:27,29/07/2021 14:57:27,29/07/2021 14:57:27,NaN,NaN,NaN,NaN,29/07/2021 14:57:27,29/07/2021 14:57:27,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29/07/2021 14:57:27,NaN
399448,29/07/2021 14:57:28,29/07/2021 14:57:28,29/07/2021 14:57:28,NaN,NaN,NaN,NaN,29/07/2021 14:57:28,29/07/2021 14:57:28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29/07/2021 14:57:28,NaN
399449,29/07/2021 14:57:29,29/07/2021 14:57:29,29/07/2021 14:57:29,NaN,NaN,NaN,NaN,29/07/2021 14:57:29,29/07/2021 14:57:29,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29/07/2021 14:57:29,NaN
399450,29/07/2021 14:57:30,29/07/2021 14:57:30,29/07/2021 14:57:30,NaN,NaN,NaN,NaN,29/07/2021 14:57:30,29/07/2021 14:57:30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29/07/2021 14:57:30,NaN


Alineo las señales que tienen periodo de muestreo distinto a 1 segundo

In [11]:
for i in range(1,12):
    data['Conc_P3-'+str(i)+' ValueY'] = dataAlign(data['Conc_P3-'+str(i)+' ValueY'],10)
    data['Conc_P3-'+str(i)+' Time'] = dataAlign(data['Conc_P3-'+str(i)+' Time'],10)
data['Conc_P1 Time'] = dataAlign(data['Conc_P1 Time'],10)
data['Conc_P1 ValueY'] = dataAlign(data['Conc_P1 ValueY'],10)
data['Q_Solvente Time'] = dataAlign(data['Q_Solvente Time'],2)
data['Q_Solvente ValueY'] = dataAlign(data['Q_Solvente ValueY'],2)
data['Q_Miscela Time'] = dataAlign(data['Q_Miscela Time'],2)
data['Q_Miscela ValueY'] = dataAlign(data['Q_Miscela ValueY'],2)
data['T_Lam_9 Time'] = dataAlign(data['T_Lam_9 Time'],10)
data['T_Lam_9 ValueY'] = dataAlign(data['T_Lam_9 ValueY'],10)
data['LT-17 Time'] = dataAlign(data['LT-17 Time'],2)
data['LT-17 ValueY'] = dataAlign(data['LT-17 ValueY'],2)
data['TT-35-GO Time'] = dataAlign(data['TT-35-GO Time'],5)
data['TT-35-GO ValueY'] = dataAlign(data['TT-35-GO ValueY'],5)

In [12]:
data[Etiquetas_time].head(20)

,Humedad_Semilla Time,T_Mat_entrada Time,T_Solvente Time,Q_Solvente Time,T_Lam_9 Time,Planta_Marcha Time,Q_Miscela Time,Vel_Extractor Time,Conc_Miscela Time,Valid_NIR_Salida_Ext Time,...,Conc_P3-5 Time,Conc_P3-6 Time,Conc_P3-7 Time,Conc_P3-8 Time,Conc_P3-9 Time,Conc_P3-10 Time,Conc_P3-11 Time,Conc_P1 Time,TT-35-GI Time,TT-35-GO Time
0,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,26/07/2021 8:51:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 7:08:02,...,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00,25/07/2021 0:00:00
1,25/07/2021 0:00:01,25/07/2021 0:00:01,25/07/2021 0:00:01,NaN,NaN,26/07/2021 8:52:00,NaN,25/07/2021 0:00:01,25/07/2021 0:00:01,25/07/2021 7:08:50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/07/2021 0:00:01,NaN
2,25/07/2021 0:00:02,25/07/2021 0:00:02,25/07/2021 0:00:02,25/07/2021 0:00:02,NaN,26/07/2021 10:40:00,25/07/2021 0:00:02,25/07/2021 0:00:02,25/07/2021 0:00:02,25/07/2021 11:32:05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/07/2021 0:00:02,NaN
3,25/07/2021 0:00:03,25/07/2021 0:00:03,25/07/2021 0:00:03,NaN,NaN,26/07/2021 10:41:00,NaN,25/07/2021 0:00:03,25/07/2021 0:00:03,25/07/2021 11:32:50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/07/2021 0:00:03,NaN
4,25/07/2021 0:00:04,25/07/2021 0:00:04,25/07/2021 0:00:04,25/07/2021 0:00:04,NaN,26/07/2021 14:05:00,25/07/2021 0:00:04,25/07/2021 0:00:04,25/07/2021 0:00:04,26/07/2021 8:51:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/07/2021 0:00:04,NaN
5,25/07/2021 0:00:05,25/07/2021 0:00:05,25/07/2021 0:00:05,NaN,NaN,26/07/2021 14:06:00,NaN,25/07/2021 0:00:05,25/07/2021 0:00:05,26/07/2021 8:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/07/2021 0:00:05,25/07/2021 0:00:05
6,25/07/2021 0:00:06,25/07/2021 0:00:06,25/07/2021 0:00:06,25/07/2021 0:00:06,NaN,27/07/2021 9:31:00,25/07/2021 0:00:06,25/07/2021 0:00:06,25/07/2021 0:00:06,26/07/2021 10:40:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/07/2021 0:00:06,NaN
7,25/07/2021 0:00:07,25/07/2021 0:00:07,25/07/2021 0:00:07,NaN,NaN,27/07/2021 22:59:00,NaN,25/07/2021 0:00:07,25/07/2021 0:00:07,26/07/2021 10:41:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/07/2021 0:00:07,NaN
8,25/07/2021 0:00:08,25/07/2021 0:00:08,25/07/2021 0:00:08,25/07/2021 0:00:08,NaN,28/07/2021 10:50:00,25/07/2021 0:00:08,25/07/2021 0:00:08,25/07/2021 0:00:08,26/07/2021 14:05:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/07/2021 0:00:08,NaN
9,25/07/2021 0:00:09,25/07/2021 0:00:09,25/07/2021 0:00:09,NaN,NaN,28/07/2021 10:51:00,NaN,25/07/2021 0:00:09,25/07/2021 0:00:09,26/07/2021 14:06:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/07/2021 0:00:09,NaN


In [13]:
# Propago valores hacia atras y hacia adelante para reemplazar los NaN
data=data.fillna(method="bfill")
data=data.fillna(method="ffill")
# Elimino etiquetas Time repetidas
index = pd.DatetimeIndex(data["Ritmo_Molienda Time"], dayfirst=True)
data=data.set_index(index)
data=data.drop(columns='Ritmo_Molienda Time')
data.index.names=['Time']
data=data.drop(columns=Etiquetas_time)

In [ ]:
# #data=data.drop(columns=Etiquetas_time)
# index = pd.DatetimeIndex(data["Ritmo_Molienda Time"], dayfirst=True)
# data=data.set_index(index)
# #data=data.drop(columns='Ritmo_Molienda Time')
# data=data.drop(columns=Etiquetas_time)
# data.index.names=['Time']

In [16]:
data = str_delete(data, ' ValueY')

In [17]:
data.head()

,Humedad_Semilla,T_Mat_entrada,T_Solvente,Q_Solvente,T_Lam_9,Planta_Marcha,Ritmo_Molienda,Q_Miscela,Vel_Extractor,Conc_Miscela,...,Conc_P3-5,Conc_P3-6,Conc_P3-7,Conc_P3-8,Conc_P3-9,Conc_P3-10,Conc_P3-11,Conc_P1,TT-35-GI,TT-35-GO
Time,,,,,,,,,,,,,,,,,,,,,
2021-07-25 00:00:00,10.580923,56.884769,65.690109,261742.859375,67.057297,1.0,203,175934.796875,0.428423,31.341852,...,21.71096,19.069359,16.825352,16.037754,15.480136,14.734509,14.381503,13.582586,58.447269,21.131729
2021-07-25 00:00:01,10.554271,56.890194,65.690109,262043.093750,67.013893,1.0,203,176040.343750,0.428423,31.355589,...,21.71096,19.069359,16.825352,16.037754,15.480136,14.734509,14.381503,13.582586,58.452694,21.131729
2021-07-25 00:00:02,10.554271,56.890194,65.690109,262043.093750,67.013893,1.0,203,176040.343750,0.428423,31.361198,...,21.71096,19.069359,16.825352,16.037754,15.480136,14.734509,14.381503,13.582586,58.441845,21.131729
2021-07-25 00:00:03,10.554271,56.890194,65.690109,262371.843750,67.013893,1.0,203,175870.515625,0.428423,31.364422,...,21.71096,19.069359,16.825352,16.037754,15.480136,14.734509,14.381503,13.582586,58.414715,21.131729
2021-07-25 00:00:04,10.554271,56.895618,65.690109,262371.843750,67.013893,1.0,203,175870.515625,0.428423,31.366344,...,21.71096,19.069359,16.825352,16.037754,15.480136,14.734509,14.381503,13.582586,58.414715,21.131729


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 399452 entries, 2021-07-25 00:00:00 to 2021-07-29 14:57:31
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Humedad_Semilla          399452 non-null  float64
 1   T_Mat_entrada            399452 non-null  float64
 2   T_Solvente               399452 non-null  float64
 3   Q_Solvente               399452 non-null  float64
 4   T_Lam_9                  399452 non-null  float64
 5   Planta_Marcha            399452 non-null  float64
 6   Ritmo_Molienda           399452 non-null  int64  
 7   Q_Miscela                399452 non-null  float64
 8   Vel_Extractor            399452 non-null  float64
 9   Conc_Miscela             399452 non-null  float64
 10  Valid_NIR_Salida_Ext     399452 non-null  float64
 11  MG_NIR_Salida_Ext        399452 non-null  float64
 12  Solvente_NIR_Salida_Ext  399452 non-null  float64
 13  LT-17                    

In [19]:
data.to_csv('To_FE_20210811_2320_3T.csv')